## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-07-20-51-17 +0000,cbc,MPs grill long ballot organizer during tense c...,https://www.cbc.ca/news/politics/mps-longest-b...
1,2025-10-07-20-45-00 +0000,wsj,White House Says Federal Workers’ Back Pay Dur...,https://www.wsj.com/politics/policy/customurl-...
2,2025-10-07-20-43-38 +0000,nyt,Tesla Reveals Cheaper Versions of Model Y and ...,https://www.nytimes.com/2025/10/07/business/te...
3,2025-10-07-20-43-08 +0000,wapo,"Gold soars past $4,000; why the record is a si...",https://www.washingtonpost.com/business/2025/1...
4,2025-10-07-20-42-50 +0000,nyt,One Hostage’s Path to Healing and Advocacy,https://www.nytimes.com/video/world/middleeast...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,49
13,shutdown,19
348,gaza,16
79,new,14
364,israel,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
276,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...,128
63,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...,113
260,2025-10-07-01-27-00 +0000,wsj,President Trump signaled a willingness to stri...,https://www.wsj.com/politics/policy/trump-open...,108
308,2025-10-06-21-45-09 +0000,latimes,Trump advisors amp up extreme rhetoric against...,https://www.latimes.com/politics/story/2025-10...,107
73,2025-10-07-17-41-13 +0000,nypost,Trump says Dems ‘have no leader’ in shutdown s...,https://nypost.com/2025/10/07/us-news/trump-ri...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
276,128,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...
283,62,2025-10-06-22-37-00 +0000,wsj,The Reserve Bank of New Zealand has establishe...,https://www.wsj.com/articles/new-zealand-centr...
63,61,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...
92,54,2025-10-07-17-01-48 +0000,latimes,Supreme Court sees a free-speech problem with ...,https://www.latimes.com/politics/story/2025-10...
120,53,2025-10-07-15-50-55 +0000,nypost,NY woman refuses to pay $440 fine to Hertz aft...,https://nypost.com/2025/10/07/business/ny-woma...
102,51,2025-10-07-16-46-07 +0000,nyt,Israel Marks Two-Year Anniversary of Oct. 7 in...,https://www.nytimes.com/2025/10/07/world/middl...
153,39,2025-10-07-14-01-04 +0000,cbc,"White House talks 'successful, positive, subst...",https://www.cbc.ca/news/politics/livestory/whi...
194,38,2025-10-07-10-00-00 +0000,nypost,Hostage Eli Sharabi was starved and beaten by ...,https://nypost.com/2025/10/07/world-news/eli-s...
142,33,2025-10-07-14-35-20 +0000,nypost,Virginia Fraternal Order of Police demands ‘un...,https://nypost.com/2025/10/07/us-news/virginia...
43,31,2025-10-07-19-20-11 +0000,nypost,"Tesla unveils long-awaited cheaper Model Y, Mo...",https://nypost.com/2025/10/07/business/tesla-u...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
